In [206]:
import pandas as pd
import numpy as np
import geopandas as gpd
import glob
from sklearn.metrics.pairwise import haversine_distances
from math import radians
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
%matplotlib inline

In [216]:
# mta ridership - aggregated on 4hr intervals

mta = pd.read_csv('subway_2021_ridership.csv')
mta.head()

,STATION,DATE_time,ENTRIES,ridership,flag
0,1 AV,2020-12-26 04:00:00,15510689,3.0,True
1,1 AV,2020-12-26 08:00:00,15510717,28.0,True
2,1 AV,2020-12-26 12:00:00,15510758,41.0,True
3,1 AV,2020-12-26 16:00:00,15510831,73.0,True
4,1 AV,2020-12-26 20:00:00,15510877,46.0,True


In [218]:
mta.shape

(626051, 5)

In [217]:
# station locations and connections

mta_loc = pd.read_csv('subway_locations_connections.csv')
mta_loc.head()

,origin_name,origin_id,origin_lat,origin_long,dest_name,dest_id,dest_lat,dest_long
0,1 AV,L06,40.730953,-73.981628,"['3 Av', 'Bedford Av']","['L05', 'L08']","[40.732849, 40.717304]","[-73.986122, -73.956872]"
1,103 ST,119,40.799446,-73.968379,"['96 St', 'Cathedral Pkwy']","['120', '118']","[40.793919, 40.803967]","[-73.972323, -73.966847]"
2,103 ST - CORONA PLAZA,706,40.749865,-73.862700,"['Junction Blvd', '111 St']","['707', '705']","[40.749145, 40.75173]","[-73.869527, -73.855334]"
3,104 ST,A63,40.681711,-73.837683,"['111 St', 'Rockaway Blvd']","['A64', 'A61']","[40.684331, 40.680429]","[-73.832163, -73.843853]"
4,110 ST,623,40.795020,-73.944250,"['116 St', '103 St']","['622', '624']","[40.798629, 40.7906]","[-73.941617, -73.947478]"


In [227]:
mta_loc.shape

(373, 8)

In [231]:
# keep only flag==True data

mta_true = mta[mta['flag'] == True]
mta_true['DATE_time'] = pd.to_datetime(mta_true['DATE_time'])
mta_true['DATE'] = mta_true['DATE_time'].dt.date

mta_rider = mta_true.groupby(by=['DATE', 'STATION'], as_index=False).sum()[['DATE', 'STATION', 'ridership']]
mta_rider.drop_duplicates(subset=['DATE', 'STATION'], inplace=True)
mta_rider.head()

mta_daily = mta_rider.merge(mta_loc, left_on='STATION', right_on='origin_name')

mta_daily.drop_duplicates(subset=['DATE', 'STATION'], inplace=True)
mta_daily.head()

,DATE,STATION,ridership,origin_name,origin_id,origin_lat,origin_long,dest_name,dest_id,dest_lat,dest_long
0,2020-12-26,1 AV,191.0,1 AV,L06,40.730953,-73.981628,"['3 Av', 'Bedford Av']","['L05', 'L08']","[40.732849, 40.717304]","[-73.986122, -73.956872]"
1,2020-12-27,1 AV,159.0,1 AV,L06,40.730953,-73.981628,"['3 Av', 'Bedford Av']","['L05', 'L08']","[40.732849, 40.717304]","[-73.986122, -73.956872]"
2,2020-12-28,1 AV,310.0,1 AV,L06,40.730953,-73.981628,"['3 Av', 'Bedford Av']","['L05', 'L08']","[40.732849, 40.717304]","[-73.986122, -73.956872]"
3,2020-12-29,1 AV,390.0,1 AV,L06,40.730953,-73.981628,"['3 Av', 'Bedford Av']","['L05', 'L08']","[40.732849, 40.717304]","[-73.986122, -73.956872]"
4,2020-12-30,1 AV,402.0,1 AV,L06,40.730953,-73.981628,"['3 Av', 'Bedford Av']","['L05', 'L08']","[40.732849, 40.717304]","[-73.986122, -73.956872]"


In [236]:
# aggregate on daily

mta_daily['DATE'] = pd.to_datetime(mta_daily['DATE'])
mta_daily['day'] = mta_daily['DATE'].dt.day_of_year
mta_daily = mta_daily[mta_daily['DATE'] > '2020-12-31']
mta_daily = mta_daily[['DATE', 'STATION', 'ridership', 'origin_lat', 'origin_long', 'day']]
mta_daily.head()

,DATE,STATION,ridership,origin_lat,origin_long,day
6,2021-01-01,1 AV,130.0,40.730953,-73.981628,1
7,2021-01-02,1 AV,199.0,40.730953,-73.981628,2
8,2021-01-03,1 AV,161.0,40.730953,-73.981628,3
9,2021-01-04,1 AV,329.0,40.730953,-73.981628,4
10,2021-01-05,1 AV,379.0,40.730953,-73.981628,5


In [237]:
# train-test-split of stations

np.random.seed(2002)

nodes_train, nodes_test = train_test_split(mta_daily.STATION.unique(), test_size=0.3)

nodes_train, nodes_val = train_test_split(nodes_train, test_size=0.3)
print(len(nodes_train), len(nodes_val), len(nodes_test))

116 50 72


In [238]:
mta_daily_train = mta_daily[mta_daily.STATION.isin(nodes_train)]
mta_daily_val = mta_daily[mta_daily.STATION.isin(nodes_val)]
mta_daily_test = mta_daily[mta_daily.STATION.isin(nodes_test)]

In [270]:
def weightFunc(dist, lamb=0):

    return np.exp(-lamb*np.array(dist))


def test_predictions(model, bestParams, data_train, data_test, weights=False):
    
    model.fit(data_train[0], data_train[1])
    
    if weights == True:
        
        dist, ind = model.kneighbors(data_test[0])
        weights = weightFunc(dist, bestParams['lambda'])
        pred_test = np.sum(weights*data_train[1][ind].reshape(weights.shape), axis=1)/np.sum(weights, axis=1)
    else:
        pred_test = model.predict(data_test[0])
    
    return r2_score(data_test[1], pred_test)

In [271]:
# simple KNN

days_r2 = []
n_days = 30
bestParams = {'lambda':None, 'best_K':None}

for day in mta_daily.day.unique()[:n_days]:
 
    X_train, y_train = mta_daily_train[mta_daily_train.day == day][['origin_lat', 'origin_long']].values, mta_daily_train[mta_daily_train.day == day]['ridership'].values
    X_val, y_val = mta_daily_val[mta_daily_val.day == day][['origin_lat', 'origin_long']].values, mta_daily_val[mta_daily_val.day == day]['ridership'].values
    X_test, y_test = mta_daily_test[mta_daily_test.day == day][['origin_lat', 'origin_long']].values, mta_daily_test[mta_daily_test.day == day]['ridership'].values
    
    best_K = None
    best_r2 = -100
    
    for k in range(1, 6):

        mod = KNeighborsRegressor(n_neighbors=k, metric='haversine')
        mod.fit(X_train, y_train)
        r2 = r2_score(y_val, mod.predict(X_val))
        if r2 > best_r2:
            best_K = k
            best_r2 = r2
            
    test_r2 = test_predictions(KNeighborsRegressor(n_neighbors=best_K, metric='haversine'), 
                                                   bestParams=bestParams, data_train=(X_train, y_train), 
                                                    data_test=(X_test, y_test))
    
    days_r2.append(test_r2)
    print('day:', day, ': ', 'best test r2:', r2)

day: 1 :  best test r2: -0.08476667510655123
day: 2 :  best test r2: -0.0905500876050136
day: 3 :  best test r2: -0.2522237368199296
day: 4 :  best test r2: -0.07120846866384856
day: 5 :  best test r2: -0.07953217501348275
day: 6 :  best test r2: -0.11376092369161506
day: 7 :  best test r2: -0.2741182686532899
day: 8 :  best test r2: -0.1863442065900196
day: 9 :  best test r2: 0.039101781255032164
day: 10 :  best test r2: -0.24316035154041415
day: 11 :  best test r2: -0.05117523415770253
day: 12 :  best test r2: -0.08087290611203324
day: 13 :  best test r2: -0.08666395702141316
day: 14 :  best test r2: -0.0464020600241426
day: 15 :  best test r2: -2.371099930784557
day: 16 :  best test r2: -0.17004417451403553
day: 17 :  best test r2: -0.19145395971191004
day: 18 :  best test r2: -0.11434670332287244
day: 19 :  best test r2: -0.06723052537498941
day: 20 :  best test r2: -0.11223087294528122
day: 21 :  best test r2: -0.07357087911616866
day: 22 :  best test r2: -0.09244570691606824
day:

In [310]:
# weighted KNN - fixed K

best_r2_weighted = []
n_days = 30
K = 4

for day in mta_daily.day.unique()[:n_days]:
    
    bestParams = {'lambda':None, 'best_K':None}
    
    X_train, y_train = mta_daily_train[mta_daily_train.day == day][['origin_lat', 'origin_long']].values, mta_daily_train[mta_daily_train.day == day]['ridership'].values
    X_val, y_val = mta_daily_val[mta_daily_val.day == day][['origin_lat', 'origin_long']].values, mta_daily_val[mta_daily_val.day == day]['ridership'].values
    X_test, y_test = mta_daily_test[mta_daily_test.day == day][['origin_lat', 'origin_long']].values, mta_daily_test[mta_daily_test.day == day]['ridership'].values
    
    best_mse = 1000000000
    best_lambda = None
    best_r2 = None
    
    mod = KNeighborsRegressor(n_neighbors=K, metric='haversine')
    mod.fit(X_train, y_train)
    dist, ind = mod.kneighbors(X_val)
    
    for lam in np.linspace(-1000, 1000, 50):

        weights = weightFunc(dist, lam)
        pred_val = np.sum(weights*y_train[ind].reshape(weights.shape), axis=1)/np.sum(weights, axis=1)
        pred_val = np.nan_to_num(pred_val, nan=0)

        mse = mean_squared_error(y_val, pred_val)
        r2 = r2_score(y_val, pred_val)

        if mse < best_mse:
            best_mse = mse
            best_lambda = lam
            best_r2 = r2
    
    bestParams['lambda'] = best_lambda
    test_r2 = test_predictions(KNeighborsRegressor(n_neighbors=K, metric='haversine'), 
                                                   bestParams=bestParams, data_train=(X_train, y_train), 
                                                    data_test=(X_test, y_test), weights=True)
    
    best_r2_weighted.append(test_r2)
    print('day:', day, ': ', 'best test r2:', r2)


day: 1 :  best test r2: -0.3359442445324998
day: 2 :  best test r2: -0.13090469623975043
day: 3 :  best test r2: -0.5311077666299984
day: 4 :  best test r2: -0.4746028280632306
day: 5 :  best test r2: -0.43726928400285936
day: 6 :  best test r2: -0.48686974210807543
day: 7 :  best test r2: -0.7489766144806485
day: 8 :  best test r2: -0.7269767935767142
day: 9 :  best test r2: -0.25040428911272006
day: 10 :  best test r2: -0.5753039282842753
day: 11 :  best test r2: -0.5410303464141322
day: 12 :  best test r2: -0.48597405489913426
day: 13 :  best test r2: -0.5824480979619948
day: 14 :  best test r2: -0.5132543535635308
day: 15 :  best test r2: -43.01277875324571
day: 16 :  best test r2: -0.3150871692583006
day: 17 :  best test r2: -0.5535304777915346
day: 18 :  best test r2: -0.4909340822719084
day: 19 :  best test r2: -0.5566160163307132
day: 20 :  best test r2: -0.6043359432400564
day: 21 :  best test r2: -0.49585426507053865
day: 22 :  best test r2: -0.35232864670305575
day: 23 :  be

In [308]:
# weighted NN around a fixed radius

best_r2_weighted_radius = []
n_days = 30

r = 0.01

for day in mta_daily.day.unique()[:n_days]:
 
    X_train, y_train = mta_daily_train[mta_daily_train.day == day][['origin_lat', 'origin_long']].values, mta_daily_train[mta_daily_train.day == day]['ridership'].values
    X_val, y_val = mta_daily_val[mta_daily_val.day == day][['origin_lat', 'origin_long']].values, mta_daily_val[mta_daily_val.day == day]['ridership'].values
    X_test, y_test = mta_daily_test[mta_daily_test.day == day][['origin_lat', 'origin_long']].values, mta_daily_test[mta_daily_test.day == day]['ridership'].values
    
    best_mse = 1000000000
    best_lambda = None
    best_r2 = None


    mod = NearestNeighbors(radius=r, metric='haversine')
    mod.fit(X_train, y_train)
    dist, ind = mod.radius_neighbors(X_val, radius=r, sort_results=True)
    dist = np.array([list(i) for i in dist])
    pad = len(max(dist, key=len))
    dist = np.array([i + [10000]*(pad-len(i)) for i in dist])
    ind = np.array([list(i) for i in ind])
    ind = np.array([i + [0]*(pad-len(i)) for i in ind])
    
    for lam in np.logspace(-4.5, 4.5):

        weights = weightFunc(dist, lam)
        pred_val = np.sum(weights*y_train[ind].reshape(weights.shape), axis=1)/np.sum(weights, axis=1)
        pred_val = np.nan_to_num(pred_val, nan=0)

        mse = mean_squared_error(y_val, pred_val)
        r2 = r2_score(y_val, pred_val)

        if mse < best_mse:
            best_mse = mse
            best_lambda = lam
            best_r2 = r2
    
    dist, ind = mod.radius_neighbors(X_test, radius=r, sort_results=True)
    dist = np.array([list(i) for i in dist])
    dist = np.array([i + [10000]*(pad-len(i)) for i in dist])
    ind = np.array([list(i) for i in ind])
    ind = np.array([i + [0]*(pad-len(i)) for i in ind])
    
    weights = weightFunc(dist, best_lambda)
    pred_test = np.sum(weights*y_train[ind].reshape(weights.shape), axis=1)/np.sum(weights, axis=1)
    pred_test = np.nan_to_num(pred_test, nan=0)
    
    print('day:', day, ': ', 'best test r2:', r2_score(y_test, pred_test))
    best_r2_weighted_radius.append(r2_score(y_test, pred_test))

day: 1 :  best test r2: -0.10591592158811203
day: 2 :  best test r2: -0.04818756057243623
day: 3 :  best test r2: 0.006915363502074778
day: 4 :  best test r2: -0.08670344283440978
day: 5 :  best test r2: -0.032087145397356354
day: 6 :  best test r2: -0.001546831041677299
day: 7 :  best test r2: -0.022560372337334877
day: 8 :  best test r2: 0.03858728121698696
day: 9 :  best test r2: 0.07303862422011509
day: 10 :  best test r2: 0.02509647299773554
day: 11 :  best test r2: -0.016568277264142894
day: 12 :  best test r2: -0.06890890755777446
day: 13 :  best test r2: -0.007482743711565876
day: 14 :  best test r2: -0.015081188077019858
day: 15 :  best test r2: -0.03842470324313574
day: 16 :  best test r2: 0.03295228329209465
day: 17 :  best test r2: 0.04485904460206169
day: 18 :  best test r2: -0.10127784832831921
day: 19 :  best test r2: -0.008879841550283096
day: 20 :  best test r2: -0.016009302505035183
day: 21 :  best test r2: 0.002001701528970168
day: 22 :  best test r2: -0.031965416178